<h2> 1. Exploratory Data Analysis </h2>

<h2> 2. Experiment & Metrics Design </h2>

The neighboring cities of Gotham and Metropolis have complementary circadian rhythms: on weekdays, Ultimate Gotham is most active at night, and Ultimate Metropolis is most active during the day. On weekends, there is reasonable activity in both cities. However, a toll bridge, with a two­way toll, between the two cities causes driver partners to tend to be exclusive to each city. The Ultimate managers of city operations for the two cities have proposed an experiment to encourage driver partners to be available in both cities, by reimbursing all toll costs.    

<h3 style="text-align:right"> 2.1 Key Measure of Success </h3>
<b>What would you choose as the key measure of success of this experiment in encouraging driver partners to serve both cities, and why would you choose this metric?</b><br />
The key measure of success I would look at is profit and if there is an increase after reimbursing all toll costs. The Ultimate managers don’t want drivers to serve both cities so the drivers can enjoy a change of scenery. By having drivers serve both cities they expand their network of cars without having to add more drivers. With an expanded network of cars, they can serve more users and increase the user experience via reduced wait times and driver availability. All of these benefits will be reflected in profit. If the initiative improves the service, then Ultimate’s profit should increase. Revenue might increase, but since they have to reimbursement the tolls that cost might eat into it, which is why the importance is on profit. 

<h3 style="text-align:right"> 2.2 Practical Experiment Design </h3>
<b>Describe a practical experiment you would design to compare the effectiveness of the proposed change in relation to the key measure of success.</b><br />

To test the effectiveness of the initiative on profits I would use an A/B test where the test group is a subset of drivers that do get reimbursed for the tolls and the control group is a subset of drivers that don’t get reimbursed for the tolls. The hypothesis for the experiment is as follows:
<ul>
    <li> Null Hypothesis $H_0$: Reimbursing driver tolls has no impact on Ultimate’s profits.</li>
    <li> Alternate Hypothesis $H_1$: Reimbursing driver tolls has an impact on Ultimate’s profits.</li>
</ul>

The partition of the drivers into control and test groups should be random, however after the split two things should be checked before the experiment begins:
<ol>
<li> There should be about the same number of drivers from both types of driver partners, those typically exclusive to Gotham and those typically exclusive to Metropolis. </li>
<li> The distribution of profits for both groups should also be about the same. If drivers in one of group have higher profit levels beforehand, that would muddy the results and interpretation of the experiment.  
</ol>

Assuming the test is successful and reimbursing tolls increases profits I would want to see the percentage increase in profits. Additionally, profits for Ultimate and profits for their drivers are two different things. They might be getting toll reimbursements but going across the bridge increases the wear and tear on the vehicle as well as the gas consumed. Additionally, if the driver goes across the bridge for a trip and then never gets a trip that sends him back over, he would be responsible for the toll to get back home when he is done working (assuming they only reimburse when the driver is on duty). Something that might be worth further investigation is if they need to reimburse all toll costs or only toll costs at certain times, like how they have “hot-zones” that pay out double the money.
The last thing that should be evaluated is how sustainable the initiative is if things change, some factors that could affect the initiative are the toll costs, the number of driver partners, and the number of riders. 




<h2> 3. Predictive Modeling </h2>